In [1]:
# Requisitos
from gensim.models import TfidfModel  # obliga a TfidfModelo mitgana normal
from gensim.utils import simple_preprocess #tokanitzar i min
from gensim.corpora import Dictionary #ns
from gensim.models import Word2Vec
import numpy as np

In [2]:
# Tipado
from typing import Tuple, List


# Part 1:  Entrenament de models

## Word2vec

### Dades

In [ ]:
from datasets import load_dataset
dataset = load_dataset("projecte-aina/catalan_general_crawling")

In [ ]:
train_data = dataset["train"]

with open("datos.txt", "w", encoding="utf-8") as file:
    for example in train_data:
        text = example["text"]
        file.write(text + "\n")


KeyboardInterrupt: 

In [ ]:
!head -c 100M datos.txt > subset_100.txt


In [ ]:
!head -c 500M datos.txt > subset_500.txt

### Entrenament

In [ ]:
from gensim.models import word2vec

In [ ]:
# Cargar el archivo de texto
sentences = word2vec.LineSentence("subset_100.txt")
model = word2vec.Word2Vec(sentences, vector_size=100, window=5, min_count=10, workers=4, epochs=25)
model.save('models/word2vec_model_100.bin')


sentences2 = word2vec.LineSentence("subset_500.txt")
model2 = word2vec.Word2Vec(sentences2, vector_size=100, window=5, min_count=10, workers=4, epochs=25)
model2.save('models/word2vec_model_500.bin')


UnicodeDecodeError: 'utf-8' codec can't decode byte 0xe7 in position 1: invalid continuation byte

In [ ]:
model100 = Word2Vec.load('models/word2vec_model_100.bin')
model500 = Word2Vec.load('models/word2vec_model_500.bin')

## Fasttext

### Carrega del model

In [ ]:
from gensim.models import fasttext
# Modelos pre-entrenados
WV_MODEL_PATH = "models/cc.ca.300.bin.gz" #model fast text 
wv_model = fasttext.load_facebook_vectors(WV_MODEL_PATH)
# Heu de guardar el model en un format compatible  #1 pas memory map
wv_model.save('cc.ca.gensim.bin')


In [ ]:
# Llavors podeu carregar el model com a mmap #2 pas memory map
from gensim.models.fasttext import FastTextKeyedVectors
ft_model = FastTextKeyedVectors.load('models/cc.ca.gensim.bin', mmap='r')


## Resultats
falta provar per model100 vs model500 (word2vec) vs ft_model(fasttext) 

In [ ]:
model100.wv['parlament'] #ja tenim una representació da cada paraula en forma de vector

In [ ]:
ft_model.doesnt_match(["cadira", "sofa", "gat", "butaca"])

In [ ]:
model500.wv.most_similar("parlament", topn=5)

In [ ]:
model100.wv.most_similar("parlament", topn=5)

In [ ]:
model500.wv.most_similar(positive=["banc", "cadira"], negative=["diners"], topn=5)


In [ ]:
model500.wv.doesnt_match(["cadira", "sofa", "gat", "butaca"])

# Part 2: Semantic Text Similarity
Este modelo utiliza gensim para convertir pares de vectores + puntuaciones en vectores (word embeddings).
Dado un dataset, infiere la puntuación de similitud entre ambas frases.

In [8]:
#importar llibreries
import pandas as pd
import tensorflow as tf
from sklearn.preprocessing import OneHotEncoder

In [ ]:
#carregar dades
from datasets import load_dataset
dataset = load_dataset("projecte-aina/sts-ca")  

In [16]:
train = dataset['train'] #datasets.dataset_dict.DatasetDict
train = train[0:len(train)] # canvia format de dataset a diccionari

validation = dataset['validation']
validation = validation[0:len(validation)] # canvia format de dataset a diccionari

test = dataset['test']
test = test[0:len(test)] # canvia format de dataset a diccionari

In [17]:
#convertir a Dataframe
df_tr = pd.DataFrame(train)
df_val = pd.DataFrame(validation)
df_ts = pd.DataFrame(test)

In [18]:
#Convertir a Llista
input_pairs = df_tr.values.tolist()
val = df_val.values.tolist()
ts = df_ts.values.tolist()

dades = input_pairs + val + ts

## Funcions comunes

### Preprocessing

In [10]:
# Cargar stopwords en Catalan
#STOPWORDS_CA = {"a", "abans", "ací", "ah", "així", "això", "al", "aleshores", "algun", "alguna", "algunes", "alguns", "alhora", "allà", "allí", "allò", "als", "altra", "altre", "altres", "amb", "ambdues", "ambdós", "anar", "ans", "apa", "aquell", "aquella", "aquelles", "aquells", "aquest", "aquesta", "aquestes", "aquests", "aquí", "baix", "bastant", "bé", "cada", "cadascuna", "cadascunes", "cadascuns", "cadascú", "com", "consegueixo", "conseguim", "conseguir", "consigueix", "consigueixen", "consigueixes", "contra", "d'un", "d'una", "d'unes", "d'uns", "dalt", "de", "del", "dels", "des", "des de", "després", "dins", "dintre", "donat", "doncs", "durant", "e", "eh", "el", "elles", "ells", "els", "em", "en", "encara", "ens", "entre", "era", "erem", "eren", "eres", "es", "esta", "estan", "estat", "estava", "estaven", "estem", "esteu", "estic", "està", "estàvem", "estàveu", "et", "etc", "ets", "fa", "faig", "fan", "fas", "fem", "fer", "feu", "fi", "fins", "fora", "gairebé", "ha", "han", "has", "haver", "havia", "he", "hem", "heu", "hi", "ho", "i", "igual", "iguals", "inclòs", "ja", "jo", "l'hi", "la", "les", "li", "li'n", "llarg", "llavors", "m'he", "ma", "mal", "malgrat", "mateix", "mateixa", "mateixes", "mateixos", "me", "mentre", "meu", "meus", "meva", "meves", "mode", "molt", "molta", "moltes", "molts", "mon", "mons", "més", "n'he", "n'hi", "ne", "ni", "no", "nogensmenys", "només", "nosaltres", "nostra", "nostre", "nostres", "o", "oh", "oi", "on", "pas", "pel", "pels", "per", "per que", "perquè", "però", "poc", "poca", "pocs", "podem", "poden", "poder", "podeu", "poques", "potser", "primer", "propi", "puc", "qual", "quals", "quan", "quant", "que", "quelcom", "qui", "quin", "quina", "quines", "quins", "què", "s'ha", "s'han", "sa", "sabem", "saben", "saber", "sabeu", "sap", "saps", "semblant", "semblants", "sense", "ser", "ses", "seu", "seus", "seva", "seves", "si", "sobre", "sobretot", "soc", "solament", "sols", "som", "son", "sons", "sota", "sou", "sóc", "són", "t'ha", "t'han", "t'he", "ta", "tal", "també", "tampoc", "tan", "tant", "tanta", "tantes", "te", "tene", "tenim", "tenir", "teniu", "teu", "teus", "teva", "teves", "tinc", "ton", "tons", "tot", "tota", "totes", "tots", "un", "una", "unes", "uns", "us", "va", "vaig", "vam", "van", "vas", "veu", "vosaltres", "vostra", "vostre", "vostres", "érem", "éreu", "és", "éssent", "últim", "ús"}
STOPWORDS_CA = {"a", "al", "el", "la", "els", "les", "de", "un", "una", "algun", "alguna" } #ha reduit la llista nomes a questes pareules

In [11]:
# Definir función de pre-procesado #tokanitza+filtre elimina stop words
def preprocess(sentence: str) -> List[str]:
    preprocessed = simple_preprocess(sentence)
    preprocessed = [token for token in preprocessed if token not in STOPWORDS_CA]
    return preprocessed

### Entrenament

In [12]:
# Definir constantes de entrenamiento
batch_size: int = 64
num_epochs: int = 64

In [13]:
# Obtener x_train e y_train
def pair_list_to_x_y(pair_list: List[Tuple[Tuple[np.ndarray, np.ndarray], int]]) -> Tuple[Tuple[np.ndarray, np.ndarray], np.ndarray]:
    """
    Otiene las matrices X_1 (N x d) , X_2 (N x d), e Y (n) a partir de listas de parejas de vectores de oraciones - Listas de (d, d, 1)
    :param pair_list:
    :return:
    """
    _x, _y = zip(*pair_list)
    _x_1, _x_2 = zip(*_x)
    return (np.array(_x_1), np.array(_x_2)), np.array(_y, dtype=np.float32, )

In [14]:
# Definir el Modelo
def build_and_compile_model(hidden_size: int = 128, embedding_size: int = 300, learning_rate: float = 0.001) -> tf.keras.Model: #ntra vector oracio 1 i 2 + misa embeding
    # Capa de entrada para los pares de vectores
    input_1 = tf.keras.Input(shape=(embedding_size,))
    input_2 = tf.keras.Input(shape=(embedding_size,))

    # Concatenar los pares de vectores
    concatenated = tf.keras.layers.Concatenate()([input_1, input_2])

    concatenated = tf.keras.layers.Dropout(0.1)(concatenated)

    # Capa oculta
    hidden = tf.keras.layers.Dense(hidden_size, activation='tanh')(concatenated)

    # Capa de salida 
    output = tf.keras.layers.Dense(1)(hidden)

    # Definir el modelo con las capas de entrada y salida
    model = tf.keras.Model(inputs=[input_1, input_2], outputs=output)

    # Compilar el modelo
    model.compile(loss='mean_absolute_error',
                  optimizer=tf.keras.optimizers.Adam(learning_rate))

    return model

In [19]:
#Per bag of words i tfidf
def per_tfidf(input_pairs):
    # Preprocesamiento de las oraciones y creación del diccionario  #definir vocabulari
    sentences_1_preproc = [preprocess(sentence_1) for sentence_1, _, _ in input_pairs] 
    sentences_2_preproc = [preprocess(sentence_2) for _, sentence_2, _ in input_pairs]
    sentence_pairs = list(zip(sentences_1_preproc, sentences_2_preproc))
    # Versión aplanada para poder entrenar el modelo
    sentences_pairs_flattened = sentences_1_preproc + sentences_2_preproc #concatena
    diccionario = Dictionary(sentences_pairs_flattened) #constructor (construim un diccionari)
    #cada paraula té associada un índex
    corpus = [diccionario.doc2bow(sent) for sent in sentences_pairs_flattened]
    modelo_tfidf = TfidfModel(corpus) #associar un index de paraula a un pes
    return diccionario, sentences_pairs_flattened,corpus,modelo_tfidf

diccionario, sentences_pairs_flattened,corpus,modelo_tfidf = per_tfidf(dades)

In [20]:
#convertir cada una de les entraddes,frase frase puntaucio a vector vector punt(funció map_pairs) [les frases en forma de vector]
#retorna vector i pes
def map_tf_idf(sentence_preproc: List[str], dictionary: Dictionary, tf_idf_model: TfidfModel) -> Tuple[List[np.ndarray], List[float]]:
    bow = dictionary.doc2bow(sentence_preproc) 
    tf_idf = tf_idf_model[bow] #entrenem el tfidf amb el bag of words obtingut
    vectors, weights = [], []
    for word_index, weight in tf_idf: 
        word = dictionary.get(word_index) #això dona la paraula original
        if word in wv_model:
            vectors.append(wv_model[word]) #vector
            weights.append(weight) #pes
    return vectors, weights

# Función para mapear una frase a su representación one-hot
def map_one_hot(sentence_preproc: List[str], encoder: OneHotEncoder) -> np.ndarray:
    sentence_encoded = encoder.transform(np.array(sentence_preproc).reshape(-1, 1))
    return sentence_encoded

def map_pairs(
        sentence_pairs: List[Tuple[str, str, float]],
        dictionary: Dictionary = None,
        one_hot_model: OneHotEncoder=None,
        tf_idf_model: TfidfModel = None,
        nlp = None,
) -> List[Tuple[Tuple[np.ndarray, np.ndarray], float]]:
    """
    Mapea los tripletes de oraciones a listas de (x, y), (pares de vectores, score)
    :param sentence_pairs:
    :param dictionary:
    :param tf_idf_model:
    :return:
    """
    # Mapeo de los pares de oraciones a pares de vectores
    pares_vectores = []
    for i, (sentence_1, sentence_2, similitud) in enumerate(sentence_pairs):
        sentence_1_preproc = preprocess(sentence_1)  #preproces
        sentence_2_preproc = preprocess(sentence_2)
        # Si usamos TF-IDF
        if tf_idf_model is not None: #es fa servir tf_idf ja que no totes les paraules tenen la mateixa rellevancia en els documents (utilitza els dos tipus de frequencia**)
            # Cálculo del promedio ponderado por TF-IDF de los word embeddings
            vectors1, weights1 = map_tf_idf(sentence_1_preproc, dictionary=dictionary, tf_idf_model=tf_idf_model, ) # asociem pes a cada una de les paraules (retorna vectors i pesos)
            vectors2, weights2 = map_tf_idf(sentence_2_preproc, dictionary=dictionary, tf_idf_model=tf_idf_model, )
            vector1 = np.average(vectors1, weights=weights1, axis=0, ) #mitjana ponderada
            vector2 = np.average(vectors2, weights=weights2, axis=0, )
        
        else: #opcio 2 -> no fem servir tf_idf
            if one_hot_model is not None:
                # Ajustamos el encoder con todas las palabras en el diccionario
                one_hot_model.fit(np.array(list(dictionary.values())).reshape(-1, 1))
                # Aplicamos OneHotEncoding a las frases preprocesadas
                vectors1 = [map_one_hot(sentence, one_hot_model) for sentence in sentence_1_preproc]
                vectors2 = [map_one_hot(sentence, one_hot_model) for sentence in sentence_2_preproc]
                vector1 = np.sum(vectors1, axis=0) 
                vector2 = np.sum(vectors2, axis=0)
            elif nlp is not None:
                sent1 = nlp(sentence_1)
                sent2 = nlp(sentence_2)
                vectors1 = [sent1[i].vector for i in range(len(sent1)) if str(sent1[i]) in nlp.vocab]
                vectors2 = [sent2[i].vector for i in range(len(sent2)) if str(sent2[i]) in nlp.vocab]

                #vectors1 = [nlp.vocab[word].vector for word in sentence_1_preproc if word in nlp.vocab]
                #vectors2 = [nlp.vocab[word].vector for word in sentence_2_preproc if word in nlp.vocab]
                vector1 = np.mean(vectors1, axis=0)
                vector2 = np.mean(vectors2, axis=0)
            else:
                # Cálculo del promedio de los word embeddings
                vectors1 = [wv_model[word] for word in sentence_1_preproc if word in wv_model]  #obtenim vectors de les dues oracions
                vectors2 = [wv_model[word] for word in sentence_2_preproc if word in wv_model]
                vector1 = np.mean(vectors1, axis=0) #fem mitjana
                vector2 = np.mean(vectors2, axis=0)
        # Añadir a la lista
        pares_vectores.append(((vector1, vector2), similitud))
    return pares_vectores
# **
## frequencia directa: numero d'aparicions en el document 
## frequencia inversa: numero de documents en que apareix /numero d'aparicions en el document
## tf_idf fa la multiplicació de tf * idf 

## One Hot

In [21]:
#Convertir a One-Hot
mapped_onehot = map_pairs(input_pairs, tf_idf_model=None, dictionary=diccionario, one_hot_model = OneHotEncoder(sparse=False)) #llsiat de tuples de vector vector, y similitud
mapped_onehot_val = map_pairs(val, tf_idf_model=None, dictionary=diccionario, one_hot_model = OneHotEncoder(sparse=False)) #llsiat de tuples de vector vector, y similitud
mapped_onehot_test = map_pairs(ts,tf_idf_model=None, dictionary=diccionario, one_hot_model = OneHotEncoder(sparse=False)) #llsiat de tuples de vector vector, y similitud

C:\Users\llumf\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\sklearn\preprocessing\_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(
C:\Users\llumf\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\sklearn\preprocessing\_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(
C:\Users\llumf\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\sklearn\preprocessing\_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed

In [29]:
with open('maps_onehot.txt',"w") as f:
    print('TRAIN',file=f)
    print(mapped_onehot,file=f)
    print('VALIDATION',file=f)
    print(mapped_onehot_val,file=f)
    print('TEST',file=f)
    print(mapped_onehot_test,file=f)

In [22]:
x_train_onehot, y_train_onehot = pair_list_to_x_y(mapped_onehot[0:])
x_val_onehot, y_val_onehot = pair_list_to_x_y(mapped_onehot_val[0:])
x_test_onehot, y_test_onehot = pair_list_to_x_y(mapped_onehot_test[0:])
x_test_onehot = (tf.squeeze(x_test_onehot[0], axis=1),tf.squeeze(x_test_onehot[1], axis=1))

In [24]:
# Preparar los conjuntos de datos de entrenamiento y validación
train_dataset_one = tf.data.Dataset.from_tensor_slices((x_train_onehot, y_train_onehot))
train_dataset_one = train_dataset_one.shuffle(buffer_size=len(x_train_onehot)).batch(batch_size)
val_dataset_one = tf.data.Dataset.from_tensor_slices((x_val_onehot, y_val_onehot))
val_dataset_one = val_dataset_one.batch(batch_size)
#test_dataset_one = tf.data.Dataset.from_tensor_slices((x_test_onehot, y_test_onehot))
#test_dataset_one = test_dataset_one.batch(batch_size)

In [25]:
train_dataset_one = train_dataset_one.map(lambda x, y: ((tf.squeeze(x[0], axis=1), tf.squeeze(x[1], axis=1)), y))
val_dataset_one = val_dataset_one.map(lambda x, y: ((tf.squeeze(x[0], axis=1), tf.squeeze(x[1], axis=1)), y))
#test_dataset_one = test_dataset_one.map(lambda x, y: ((tf.squeeze(x[0], axis=1), tf.squeeze(x[1], axis=1)), y))

Instructions for updating:
Lambda fuctions will be no more assumed to be used in the statement where they are used, or at least in the same block. https://github.com/tensorflow/tensorflow/issues/56089


In [26]:
# Construir y compilar el modelo
model = build_and_compile_model(embedding_size=len(diccionario))

# Entrenar el modelo
model.fit(train_dataset_one, epochs=num_epochs, validation_data=val_dataset_one)

Epoch 1/64
33/33 [==============================] - 3s 58ms/step - loss: 1.3238 - val_loss: 0.9089
Epoch 2/64
33/33 [==============================] - 2s 58ms/step - loss: 0.8413 - val_loss: 0.8856
Epoch 3/64
33/33 [==============================] - 2s 63ms/step - loss: 0.6693 - val_loss: 0.8532
Epoch 4/64
33/33 [==============================] - 2s 63ms/step - loss: 0.5657 - val_loss: 0.9927
Epoch 5/64
33/33 [==============================] - 2s 55ms/step - loss: 0.5724 - val_loss: 0.9451
Epoch 6/64
33/33 [==============================] - 2s 58ms/step - loss: 0.5810 - val_loss: 0.8362
Epoch 7/64
33/33 [==============================] - 2s 57ms/step - loss: 0.5115 - val_loss: 0.8696
Epoch 8/64
33/33 [==============================] - 2s 59ms/step - loss: 0.5305 - val_loss: 0.8211
Epoch 9/64
33/33 [==============================] - 2s 56ms/step - loss: 0.4696 - val_loss: 0.9027
Epoch 10/64
33/33 [==============================] - 2s 59ms/step - loss: 0.4374 - val_loss: 0.9303
Epoch 11/

In [30]:
model.save('mlp_onehot.bin')

INFO:tensorflow:Assets written to: mlp_onehot.bin\assets


INFO:tensorflow:Assets written to: mlp_onehot.bin\assets


In [31]:
from scipy.stats import pearsonr
# Obtener las predicciones del modelo para los datos de prueba. En este ejemplo vamos a utilizar el corpus de training.
y_pred = model.predict(x_test_onehot)
print(y_pred)
# Calcular la correlación de Pearson entre las predicciones y los datos de prueba
correlation, _ = pearsonr(y_pred.flatten(), y_test_onehot.flatten())
# Imprimir el coeficiente de correlación de Pearson
print(f"Correlación de Pearson: {correlation}")


16/16 [==============================] - 0s 6ms/step
[[2.8985887 ]
 [3.1486604 ]
 [1.6135013 ]
 [2.3479908 ]
 [2.3020751 ]
 [2.95383   ]
 [1.8273206 ]
 [2.6824567 ]
 [2.5674171 ]
 [2.5127347 ]
 [2.1076992 ]
 [2.146153  ]
 [2.530677  ]
 [2.1120076 ]
 [3.0523064 ]
 [2.4571414 ]
 [2.756591  ]
 [2.4207056 ]
 [1.7207773 ]
 [2.6669133 ]
 [1.7423093 ]
 [1.6263037 ]
 [1.8231924 ]
 [1.9954944 ]
 [1.7806822 ]
 [1.9790319 ]
 [1.366075  ]
 [2.1870227 ]
 [2.2005222 ]
 [2.7183478 ]
 [1.7238982 ]
 [2.3284452 ]
 [2.2930439 ]
 [2.519018  ]
 [3.1453032 ]
 [1.6507316 ]
 [1.7522564 ]
 [2.422029  ]
 [1.8198595 ]
 [3.3635283 ]
 [2.4597185 ]
 [1.881509  ]
 [2.6133652 ]
 [2.3381736 ]
 [2.0444922 ]
 [3.4486687 ]
 [2.6873996 ]
 [1.3773041 ]
 [1.5094054 ]
 [2.221162  ]
 [2.2095456 ]
 [2.1732333 ]
 [2.5654547 ]
 [0.9261941 ]
 [3.6719825 ]
 [1.7840106 ]
 [2.5781229 ]
 [1.9575707 ]
 [1.6963739 ]
 [2.8109043 ]
 [3.2292912 ]
 [3.2718205 ]
 [2.9408493 ]
 [2.0823271 ]
 [2.8426147 ]
 [2.3144884 ]
 [1.4390489 ]
 [2.27144

In [40]:
# Podemos evaluar el modelo si sólo utilizamos COS similarity. (Depende completamente de los Word Embeddings)
from scipy.stats import pearsonr
from scipy import spatial

y_pred_baseline = []

vs1, vs2 = x_test_onehot
for v1,v2 in zip(vs1,vs2):
    d= 1.0 - spatial.distance.cosine(v1, v2)
    y_pred_baseline.append(d)
# Calcular la correlación de Pearson entre las predicciones y los datos de prueba
correlation, _ = pearsonr(np.array(y_pred_baseline), y_test_onehot.flatten())
# Imprimir el coeficiente de correlación de Pearson
print(f"Correlación de Pearson: {correlation}") 

Correlación de Pearson: 0.6151985672634542


## Word2vec

In [41]:
from scipy.stats import pearsonr

In [42]:
def executar_testejar(model_name, one_hot_model: OneHotEncoder=None, tf_idf_model: TfidfModel = None):
    wv_model = eval(model_name)
    mapped = map_pairs(input_pairs, tf_idf_model=tf_idf_model, dictionary=diccionario, one_hot_model=one_hot_model) 
    mapped_val = map_pairs(val, tf_idf_model=tf_idf_model, dictionary=diccionario, one_hot_model=one_hot_model)
    mapped_ts = map_pairs(ts, tf_idf_model=tf_idf_model, dictionary=diccionario, one_hot_model=one_hot_model) 

    # Obtener las listas de train y test
    x_train, y_train = pair_list_to_x_y(mapped[0:])
    x_val, y_val = pair_list_to_x_y(mapped_val[0:])
    x_ts, y_ts = pair_list_to_x_y(mapped_ts[0:])
    x_ts = (tf.squeeze(x_ts[0], axis=1),tf.squeeze(x_ts[1], axis=1))

    # Preparar los conjuntos de datos de entrenamiento y validación
    train_dataset = tf.data.Dataset.from_tensor_slices((x_train, y_train))
    train_dataset = train_dataset.shuffle(buffer_size=len(x_train)).batch(batch_size)
    val_dataset = tf.data.Dataset.from_tensor_slices((x_val, y_val))
    val_dataset = val_dataset.batch(batch_size)
    #test_dataset_one = tf.data.Dataset.from_tensor_slices((x_test_onehot, y_test_onehot))
    #test_dataset_one = test_dataset_one.batch(batch_size)

    train_dataset = train_dataset.map(lambda x, y: ((tf.squeeze(x[0], axis=1), tf.squeeze(x[1], axis=1)), y))
    val_dataset = val_dataset.map(lambda x, y: ((tf.squeeze(x[0], axis=1), tf.squeeze(x[1], axis=1)), y))
    #test_dataset_one = test_dataset_one.map(lambda x, y: ((tf.squeeze(x[0], axis=1), tf.squeeze(x[1], axis=1)), y))

    # Construir y compilar el modelo
    model = build_and_compile_model(embedding_size=len(diccionario))

    # Entrenar el modelo
    model.fit(train_dataset, epochs=num_epochs, validation_data=val_dataset)
    
    nom = 'mlp' + model_name
    if tf_idf_model is not None:
        nom += 'tfidf'
    nom += '.bin'

    model.save(nom)

    y_pred = model.predict(x_ts)
    print(y_pred)
    # Calcular la correlación de Pearson entre las predicciones y los datos de prueba
    correlation, _ = pearsonr(y_pred.flatten(), y_ts.flatten())
    # Imprimir el coeficiente de correlación de Pearson
    print(f"Correlación de Pearson embeding: {correlation}")

    y_pred_baseline = []

    vs1, vs2 = x_test
    for v1,v2 in zip(vs1,vs2):
        d= 1.0 - spatial.distance.cosine(v1, v2)
        y_pred_baseline.append(d)
    # Calcular la correlación de Pearson entre las predicciones y los datos de prueba
    correlation, _ = pearsonr(np.array(y_pred_baseline), y_test.flatten())
    # Imprimir el coeficiente de correlación de Pearson
    print(f"Correlación de Pearson: {correlation}") 

In [43]:
executar_testejar('model100',tf_idf_model=modelo_tfidf)
executar_testejar('model500',tf_idf_model=modelo_tfidf)
executar_testejar('model100')
executar_testejar('model500')

NameError: name 'model100' is not defined

### 100 MB amb Mean

In [ ]:
#Convertir a vector amb model que tenguem guardat a la variable wv_model 
# Imprimir los pares de vectores y la puntuación de similitud asociada
wv_model = model100
mapped = map_pairs(input_pairs, tf_idf_model=None, dictionary=diccionario, ) 
mapped_val = map_pairs(val, tf_idf_model=None, dictionary=diccionario, )
mapped_ts = map_pairs(ts, tf_idf_model=None, dictionary=diccionario, ) 

# Obtener las listas de train y test
x_train, y_train = pair_list_to_x_y(mapped[0:])
x_val, y_val = pair_list_to_x_y(mapped_val[0:])
x_ts, y_ts = pair_list_to_x_y(mapped_ts[0:])

# Preparar los conjuntos de datos de entrenamiento y validación
train_dataset = tf.data.Dataset.from_tensor_slices((x_train, y_train))
train_dataset = train_dataset.shuffle(buffer_size=len(x_train)).batch(batch_size)
val_dataset = tf.data.Dataset.from_tensor_slices((x_val, y_val))
val_dataset = val_dataset.batch(batch_size)

## spaCy

In [54]:
import spacy
nlp = spacy.load("ca_core_web_sm")

NameError: name '_C' is not defined

In [53]:
mapped_spacy = map_pairs(input_pairs,nlp=nlp)
mapped_spacy_val = map_pairs(val,nlp=nlp)
mapped_spacy_test = map_pairs(ts,nlp=nlp)

NameError: name 'nlp' is not defined

## RoBERTa

## RoBERTa fine-tuned

## Resultats model ?

In [68]:
# Construir y compilar el modelo
model = build_and_compile_model()

# Entrenar el modelo
model.fit(train_dataset_one, epochs=num_epochs, validation_data=val_dataset_one)

Epoch 1/64
33/33 [==============================] - 1s 5ms/step - loss: 1.7734 - val_loss: 1.5098
Epoch 2/64
33/33 [==============================] - 0s 3ms/step - loss: 0.8642 - val_loss: 1.0462
Epoch 3/64
33/33 [==============================] - 0s 3ms/step - loss: 0.7927 - val_loss: 0.9397
Epoch 4/64
33/33 [==============================] - 0s 3ms/step - loss: 0.7361 - val_loss: 0.8225
Epoch 5/64
33/33 [==============================] - 0s 5ms/step - loss: 0.6796 - val_loss: 0.7539
Epoch 6/64
33/33 [==============================] - 0s 3ms/step - loss: 0.6461 - val_loss: 0.7374
Epoch 7/64
33/33 [==============================] - 0s 3ms/step - loss: 0.6181 - val_loss: 0.7376
Epoch 8/64
33/33 [==============================] - 0s 3ms/step - loss: 0.6028 - val_loss: 0.7587
Epoch 9/64
33/33 [==============================] - 0s 3ms/step - loss: 0.5974 - val_loss: 0.7697
Epoch 10/64
33/33 [==============================] - 0s 3ms/step - loss: 0.5919 - val_loss: 0.7735
Epoch 11/64
33/33 [

In [161]:
from scipy.stats import pearsonr
# Obtener las predicciones del modelo para los datos de prueba. En este ejemplo vamos a utilizar el corpus de training.
y_pred = model.predict(x_ts)
print(y_pred)
# Calcular la correlación de Pearson entre las predicciones y los datos de prueba
correlation, _ = pearsonr(y_pred.flatten(), y_ts.flatten())
# Imprimir el coeficiente de correlación de Pearson
print(f"Correlación de Pearson: {correlation}")


16/16 [==============================] - 0s 2ms/step
[[2.959906  ]
 [2.7287447 ]
 [2.0964587 ]
 [3.0246198 ]
 [2.1007998 ]
 [2.508594  ]
 [2.5175948 ]
 [3.1220758 ]
 [2.5504136 ]
 [2.4958787 ]
 [2.809256  ]
 [1.9340656 ]
 [2.5192623 ]
 [2.8160038 ]
 [1.9605311 ]
 [2.897645  ]
 [2.7465363 ]
 [2.581646  ]
 [2.1929758 ]
 [2.7899146 ]
 [3.1657722 ]
 [2.3944235 ]
 [1.9932104 ]
 [2.0409021 ]
 [1.9779159 ]
 [2.3952172 ]
 [1.6788062 ]
 [2.4390035 ]
 [2.7964895 ]
 [2.7314947 ]
 [2.4943647 ]
 [2.04224   ]
 [3.2449856 ]
 [2.4676743 ]
 [2.6310506 ]
 [1.9656539 ]
 [2.744923  ]
 [3.1324172 ]
 [2.5188184 ]
 [2.5353508 ]
 [2.5729127 ]
 [2.3699045 ]
 [2.5956416 ]
 [2.7515478 ]
 [2.3629327 ]
 [2.7782135 ]
 [2.3407838 ]
 [1.6200382 ]
 [1.5239347 ]
 [2.957579  ]
 [2.1451528 ]
 [2.4804516 ]
 [2.7493467 ]
 [2.327356  ]
 [2.8506825 ]
 [2.685415  ]
 [2.6138704 ]
 [1.9531614 ]
 [2.3962452 ]
 [2.822188  ]
 [2.7642736 ]
 [2.6101506 ]
 [2.810471  ]
 [2.5638103 ]
 [2.5347822 ]
 [2.37038   ]
 [2.5477252 ]
 [1.63549

In [162]:
list(zip(y_pred, y_train))

[(array([2.959906], dtype=float32), 3.5),
 (array([2.7287447], dtype=float32), 1.25),
 (array([2.0964587], dtype=float32), 3.67),
 (array([3.0246198], dtype=float32), 2.25),
 (array([2.1007998], dtype=float32), 2.0),
 (array([2.508594], dtype=float32), 2.75),
 (array([2.5175948], dtype=float32), 2.67),
 (array([3.1220758], dtype=float32), 2.5),
 (array([2.5504136], dtype=float32), 2.5),
 (array([2.4958787], dtype=float32), 3.0),
 (array([2.809256], dtype=float32), 3.0),
 (array([1.9340656], dtype=float32), 1.0),
 (array([2.5192623], dtype=float32), 2.0),
 (array([2.8160038], dtype=float32), 4.0),
 (array([1.9605311], dtype=float32), 3.0),
 (array([2.897645], dtype=float32), 2.75),
 (array([2.7465363], dtype=float32), 2.0),
 (array([2.581646], dtype=float32), 3.0),
 (array([2.1929758], dtype=float32), 2.0),
 (array([2.7899146], dtype=float32), 3.75),
 (array([3.1657722], dtype=float32), 2.75),
 (array([2.3944235], dtype=float32), 1.67),
 (array([1.9932104], dtype=float32), 4.5),
 (array

hem de fer més complicat aquest model

afegir complexitat a la xarxa neuronal

la tasca es massa complicada pel q tenim (no te prou informació)

footoooss (tambe ha canviat loss a mean_square_error)

hem d'intentar augmentar la correlació de pearson

la puntuació que s'obté és bastant baixa (amb transformers tochooos és 0.7)

pero la correlacio que podem obtenir amb el train és molt baixa ja que la informació no és suficient

Què hem de fer: intentar introduir tota la informació disponible a la xarxa


sequencia de paraules: a,b,c,d,e 
el que estem fent ara mateix és:
convertim cada paraula en vectors

v0, v1, v2, v3, v4

això ho este

ve, VI, v2, v3, v4
aixà ho esteim fent amb fast text dimensio vectors = 300


per tant, estem perdent molta informació al reduir la frase en un sol vector

hem de fer que la xarxa pugui rebre tots aquests vectors. Com ho podem fer?
language model basat en Finestres(concatenam tots els vectors enlloc de pasar la Mitjana)

llavors multiplicariem la nova z = [v0:v1:v2:v3:v4] per la matriu de weights

mida de nova z0 = n * 300 (n mida de la finestra/frase)
n hauria de ser 30 pq es la longitut máxima de les frases

llavors mida de W = h * |z0|
h és la mida de la hidden layer 

segueix sent un model de finestra encara q sempre té la mateixa mida
aquí hi ha problemas:
- tenim 2000 parelles de relacions. Per tant hem d'entrenar 30*300 parametres per 2000 exemples
- cada un dels weights aprendra la posicio de l'entrada (futbol a posicio 3 no apren fubtbol en una altra posició)
               n
       ----------------
       |              |
       |              |
z0 =   |              | d
       |              |
       |              |
       ----------------

Si enlloc de donar la z0 directament, la passam abans per una funció, esteim reduint les dimensions
multiplicar z0 per una matriu de projecció MATRIU dimensions (n*d) x 30
llavors tendrem un vector de dimensió 30
si feim mitjana en direcció 

problema: weigths diferents en les diferents posicions

baseline fa global average pooling (mitjana)

només substituim sa ultima capa hidden
ara podriem aplicar diferents funcions: 

- convolucionals (filtres concolucinals -> tenim matriu gran i la multiplicam per una més petita)
                  multiplicam per aquesta matriu més petita i obtenim un valor per cada bloc (aquest valor serà nsq d'activació)
                  (ho farem tipo 'finestra' aportacio de llum per entendrem a mi mateixa)
                  la mida sera n-1 en aquest cas
                  padding o mirror padding (no fer la ultima o ficar 0 despres de la ultima)
                  stride -> podem aplicar el filtre  ignorant les posicions impars (llavors mida nova = n/2) (moviment/desplaçament)
                       normalment en el pooling, no al aplicar filtre
            noltros aplicarem es filtres per paraules (és a dir amb dimensió d * x). Això dona un vector de mida n o n-1
            (finestra de x paraules)
            si apliquem més d'un filtre (per exemple 50 filtres) -> queda una matriu de 50 * n (ja que ficam es resultat de cada filtre seguit)  
            despres voldrem un sol vector no matriu, llavors fariem un pooling i obtendriem un vector 50*1
            realment sera contextual
- podem aplicar una xarxa recurrent (CONCLUSIO, NO FUNCIONA)
        agafem només una de les posicions (nomes una de les paraules) v4 per exemple
        com tenim embedings distribucionals, la representacio de cada paraula no depen del context.
        si v4 depengues de les altres paraules, llavors podria ser útil ja que tendria informació de les altres paraules
              multiplicar per la matriu de pesos (realment no es així ja que tambe multiplicam per la sortida de la posició anterior)
              h0 = W[v0*h-1]
              h1 = W[v1*h0] 

              llavors h4 = W[v4*h3]  #així tenim ja vectors contextuals ja que multiplicam sempre per l'anterior (és recurrent)
              podriem utilitzar h4 com a vector q representi tota la oració
              Ara falta la funció d'activavió !
       a efectes practics tenim n capes (si alguna de les derivades es propera a 0 no funciona be pq es gradient perd pes)
        
- atenció #esta a tenerflow: idea es:  relativament sembalnt.
       esteim fent mitjana ponderada de totes les paraules
       pq no fem una mitjana ponderada q depengui de la paraula
       Calcular els weights per calcular la mitjana ponderada

       h4 = alpha0 * v0 + alpha1 * v1 ... 
              alpha0 depen de v0 i v4
              alpha1 depen de v1 i v4
       llavors h4 te la informació de tota la seqüència

diferencies entre models: cap (mitjana se fa fora o dins de la xarxa)

QUE POLLES ESTA DIENTTTTTTTTT AAAAAAAAAAA!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
 ja, jo q faig xndl m'estic perdent no imagin tu
 :( ALAAA el ( esta vermell trist :() xiii, té es llavis pintats
idea al aplicara a text  conbu:  per aplicar nigrams

                

reduir dimensionalitat?



spoiler: parlarem de transformers (gpt i Bart) 'Atention is all you need' -> atention + feed forward i jastaaa



que es paddisn??????????????????????????

stride